In [2]:
#!pip install citipy

In [3]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
import time
from scipy.stats import linregress
# Import api key
from config import api_key
# Import citipy for city names by long. and lat.
from citipy import citipy

In [4]:
# Define ranges for longitude and latitude
latitude_range = (-90,90)
longitude_range = (-180,180)

In [16]:
# Create lists for coordinates and cities
lat_long_coord = []
cities = []

# Create random latitude and longitude combinations
latitude = np.random.uniform(latitude_range[0], latitude_range[1], size=1500)
longitude = np.random.uniform(longitude_range[0], longitude_range[1], size=1500)
lat_long_coord = zip(latitude, longitude)

# Build a for loop to find city names for random coordinates
for lat_long in lat_long_coord:
    city = citipy.nearest_city(lat_long[0], lat_long[1])
    
    # Add the city to a list if it is unique
    if city not in cities:
        cities.append(city)

In [17]:
# Check length of the list
len(cities)

642

In [ ]:
# Create base url
base_url = f"http://api.openweathermap.org/data/2.5/weather?appid={api_key}"
# define units
units = "imperial"

# Build query url with units
query_url = f"{base_url}&units={units}&q="

In [ ]:
# Create lists for results
temperature = []
humidity = []
cloudiness = []
wind_speed = []

In [ ]:
# Loop through cities to find results
for city in cities:
    
    # Create url for city
    full_url = f"{query_url}{city}"
    
    try:
    # Find json response for city
        response = requests.get(full_url).json()
        city_number = response["id"]
        
        # Find data to append for each city
        temp = response["main"]["temp"]
        humid = response["main"]["humidity"]
        cloud = response["clouds"]["all"]
        wind = response["wind"]["speed"]
        
         # Append results
        print(f"{city} found. City Number: {city_number}. Appending results.")
        
        temperature.append()
        humidity.append()
        cloudiness.append()
        wind_speed.append()
        cities_reduced.append()
    except:
        print(f"{city} not found.")
        